# Exercise 4 -Computer Vision


### 4.1 - Face Detection and Tracking
In this task you will implement face detection and tracking using OpenCV. Specifically we are utilizing Cascade classifiers which implements Viola-Jones detection algorithm.

**Reference**
- [OpenCV documentation on cascade classifier](https://docs.opencv.org/master/db/d28/tutorial_cascade_classifier.html)

### 4.1.1
Execute the code below to initiate the cascadee classifier and the utility libraries

In [1]:
import cv2
import time
import numpy as np
import time
import imutils

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 

### 4.1.2

Similar to Task 3, the first step is to obtain a frame from video file and pre-processing it. 

**Your task**

Complete prep() function below which performs following using opencv and imutils libraries. The steps already implemented are marked with a tick "✓"

- [x] Takes a frame from video feed as the input
- [ ] Resize the frame while protecting the aspect ratio (width = 600) 
- [ ] Flip the image
- [ ] Convert the frame to grayscale image
- [x] Return grayscale image and resized image 

**References**

- [imutils documentation](https://github.com/PyImageSearch/imutils#imutils)
- [Fip an array with OpenCV](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#gaca7be533e3dac7feb70fc60635adf441)
- [color conversion with OpenCV](https://docs.opencv.org/4.x/d8/d01/group__imgproc__color__conversions.html)

In [2]:
def prep(img):
    ## ToDo 4.1.2
    #  1. resize using  imutils.resize()
    img = imutils.resize(img, width=600)
    
    #  2. flip image vertically using cv2.flip()
    img = cv2.flip(img, 1)
    
    #  3. convert to gray color using cv2.cvtColor()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    return gray, img    

### 4.1.3

In 4.1.1 we initialized an instance of cascade classifier. Tracking a face can be broken down into 3 steps as below

1. Detect Faces and ROIs

   The cascade classifier has a member function which can detect faces of multiple scales in a given image. The area where a face is detected becomes a region of interest (ROI) for extracting meaningful information. 

    **References** : 
    [Multiscale face detection member function of cascade classifier](https://docs.opencv.org/3.4/d1/de5/classcv_1_1CascadeClassifier.html#a90fe1b7778bed4a27aa8482e1eecc116)


2. Extract trackable features 

    Shi-Tomasi Corner Detector is an implementation in openCV which extracts information from the ROI input. The extracted information are points on the face which are are trackable across a sequence of moving frames (a video).

    **References** : 
    [OpenCV Trackable feature extraction function(Shi-Tomasi Corner Detector)](https://docs.opencv.org/4.5.2/d4/d8c/tutorial_py_shi_tomasi.html)


3. Calculate the optical flow

    These trackable points are used to calculate the optical flow of the faces with calcOpticalFlowPyrLK() function. The tracking is visualized via OpenCV drawing tools.

    **References** : 
    - [Optical Flow calculation](https://docs.opencv.org/4.5.3/d4/dee/tutorial_optical_flow.html)
    - [OpenCV drawing functions](https://docs.opencv.org/4.5.2/dc/da5/tutorial_py_drawing_functions.html)

**Your task**

Complete the function which perfoms following

- [x] Takes grayscale image and resized image as the input
- [x] Detect faces in graycale image using cascade classifier. detectMultiscale() function returns detected faces as rectangles ( Top left x coordinate, Top left y coordinate, width, height)
- [ ] Draw a rectangle around detected faces using OpenCV drawing functions
- [ ] Slice a region of interest (ROI) from grayecale image corresponding to the detections
- [x] Extract good features to track (p0), from OpenCV goodFeaturesToTrack() function.
- [ ] Convert the array p0 from current format [[[x1,y1],[x2,y2],....]] to --> [[x1,y1],[x2,y2],....]. Tip : print p0 to observe current format
- [ ] The points are located with respect to the ROI coordinates. Convert them to image coordinates

In [3]:
def get_trackable_points(gray,img):
    # 1. Use the Cascade classifier to detect faces in the grayscale image
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    
    # 2. Drawing Rectangles and Extracting ROIs
    
    # 2.1 Initialize an empty list to store trackable points
    p0=[]
    
    # 2.2 Check if the faces are detected
    if len(faces) != 0:
        ## ToDO 4.1.3
        # for (x,y,w,h) in faces:
        #   draw rectang
        #   slice ROI      
        
        for (x, y, w, h) in faces:
            # 2.3 Draw a blue rectangle around the detected face
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            # 2.4 Slice the grayscale image to extract ROI corresponding to the detected face
            roi_gray = gray[y:y+h,x:x+w]
            
            # 2.5 Extract good features to track (p0) from each ROI
            p0_roi = cv2.goodFeaturesToTrack(roi_gray, maxCorners=70, qualityLevel=0.001, minDistance=5)
            p0.append(p0_roi)
            
        # ToDO 4.1.3
        #  covert fromat of p0 to [[x1,y1],[x2,y2],....] 
        #  convert points from ROI to image coordinates
        
        # 3. Convert the array p0 from the format [[[x1, y1]], [[x2, y2]], ...] to [[x1, y1], [x2, y2], ...]
        # Use list comprehension that flattens a list of lists
        p0 = [item[0] for sublist in p0 for item in sublist]

        # 4. Convert the points from ROI coordinates to image coordinates
        p0 = [[x + x1, y + y1] for (x1, y1) in p0]
        
        # convert into float
        p0 = np.float32(p0)
   
    return p0, faces, img

**Your task**

Complete the do_track_face() function which perfoms following

- [x] Usecv2.calcOpticalFlowPyrLK()to calculate the optical flow for tracking face
- [ ] Select the valid points from p1. Note that  isFound == 1 for valid points 
- [ ] Return the valid points as a numpy array

In [4]:
def do_track_face(gray_prev, gray, p0):
    # calculate the optical flow for tracking face
    p1, isFound, err = cv2.calcOpticalFlowPyrLK(gray_prev, gray, p0, 
                                                            None,
                                                            winSize=(31,31),
                                                            maxLevel=10,
                                                            criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03),
                                                            flags=cv2.OPTFLOW_LK_GET_MIN_EIGENVALS,
                                                            minEigThreshold=0.00025)
    
    ## ToDo 4.1.3 - Select valid points from p1
    
    # Convert isFound to a 1D array (i.e., removing unnecessary dimensions)
    isFound = np.squeeze(isFound)
    
    # Select only the points that are found and considered valid
    valid_points = p1[isFound==1]

    # return a numpy array of selected points from p1
    return np.array(valid_points)
    

# ROS Node for Task 5

In [5]:
import sys
import rclpy
from rclpy.node import Node

import cv2
from cv_bridge import CvBridge

import numpy as np
from sensor_msgs.msg import Image, CameraInfo

# bridging
bridge = CvBridge()
FRAME_RATE = 30

class FaceDetectionTrackingNode(Node):
    def __init__(self):
        # Constructor
        super().__init__('Face_Detection_Tracking')

        # Declare parameters
        self.frame = []
        self.p0 = [] # previous points
        self.p1 = []
        self.prev = 0
        self.gray_prev = None  # previous frame

        time_period = 0.1  # sec (for timer callback)
        
        # Make subscriber
        self.subscriber_get_image = self.create_subscription( Image,'/image_raw', self.get_image,10)
        
        # Make timer for callback 
        self.timer = self.create_timer(time_period, self.detect_track_face)

    def terminate(self):
        self.get_logger().info('Video is being terminated!')
    
    def get_image(self, msg):
        """
        callback function for sunscribing to raw_image topic 
        """
        self.get_logger().info("Image callback function triggered!")
        
        # Convert ros2 image into OpenCV image
        self.frame =  bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
        
        return self.frame
    
    def detect_track_face(self):
        """
        Callback function for face detection and tracking
        """
            
        time_elapsed = time.time() - self.prev

        if time_elapsed > 1./FRAME_RATE:

            self.prev = time.time()
            gray, img = prep(self.frame)

            self.p0, self.faces, img = get_trackable_points(gray,img)
            self.gray_prev = gray.copy()

            for (x,y,w,h) in self.faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                self.p1 = do_track_face(self.gray_prev, gray, self.p0)
            
            for i in self.p1:
                cv2.drawMarker(img, (int(i[0]), int(i[1])),[255,0,0],0)
            self.p0 = self.p1
                       
            cv2.imshow('Video feed', img)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord("q"):
                cv2.destroyAllWindows()
                self.stop_stream()
        

def main(args=None):
    try:
        rclpy.init(args=None)
        face_detection_tracking = FaceDetectionTrackingNode()
        rclpy.spin(face_detection_tracking)

    except KeyboardInterrupt:
        image_subscriber.terminate()
        image_subscriber.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()


[INFO] [1704111385.394688191] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.401352781] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.609001262] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.638907090] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.643788185] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.663334290] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.739577764] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.787690027] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.854294301] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.963592743] [Face_Detection_Tracking]: Image callback function triggered!
[INFO] [1704111385.990784878] [Face_Detection_Tracking]: Image callback function

AttributeError: 'FaceDetectionTrackingNode' object has no attribute 'stop_stream'